In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from utils.data_loader import Dataset
from utils.helpers import * 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR 
import xgboost
from sklearn.ensemble import RandomForestRegressor

## TODO:
1. create a benchmark for comparing the results 
2. measure the importance of features based on 

In [2]:
training_filenames = [ './data/training_fluid_intelligenceV1.csv', './data/btsv01.txt', './data/cc_training_v2.csv', \
                     'data/training_data_means.csv', 'data/training_data_entropy.csv',  'data/training_data_stdevs.csv',]
validation_filenames=[ './data/validation_fluid_intelligenceV1.csv', './data/btsv01.txt', './data/cc_validation_v2.csv', \
                     'data/validation_data_means.csv', 'data/validation_data_entropy.csv', 'data/validation_data_stdevs.csv']

cols_to_drop = ['btsv01_id', 'interview_date', 'collection_id', 'dataset_id', 'collection_title', \
                'src_subject_id', 'gender']

label_col = 'residual_fluid_intelligence_score'

training = Dataset(training_filenames, cols_to_drop, label_col)
validation = Dataset(validation_filenames, cols_to_drop, label_col)

#train - 3723 vs 3739
#val - 411 vs 415

/Users/admin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2885: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
scaler = StandardScaler()
train_data = scaler.fit_transform(training.data)
val_data = scaler.transform(validation.data)

In [4]:
#generating custom columns
is_frontal = np.array([x for x in range(train_data.shape[1]) if 'frontal' in training.meta_data['final_dataset']['columns'][x]])
is_suptent = np.array([x for x in range(train_data.shape[1]) if 'supratentorium' in training.meta_data['final_dataset']['columns'][x]])
structures_to_delete = ['thalamus', 'caudate', 'putamen', 'pallidum', 'volume', 'wm', 'supratentorium', 'csf']
cortex_indices = []
for i,column in enumerate(training.meta_data['final_dataset']['columns']):
    curr_deletes = []
    for name in structures_to_delete: 
        if name in column:
            curr_deletes.append(name)
    if len(curr_deletes)==0 and i>=0:
        cortex_indices.append(i)

cortex_indices = np.array(cortex_indices)

def generate_frontal_ratio(frontal_inds, reference_inds, data):
    coefs = []
    for observation in data:
        frontal_volume = np.sum(observation[frontal_inds])
        reference_volume = np.sum(observation[reference_inds])
        coefs.append(frontal_volume/reference_volume)
    return np.array(coefs).reshape(-1,1)


frontal_suptent_train = scaler.fit_transform(generate_frontal_ratio(is_frontal, is_suptent, train_data))
frontal_suptent_val = scaler.transform(generate_frontal_ratio(is_frontal, is_suptent, val_data))
frontal_cortex_train= scaler.fit_transform(generate_frontal_ratio(is_frontal, cortex_indices, train_data))
frontal_cortex_val = scaler.transform(generate_frontal_ratio(is_frontal, cortex_indices, val_data))

#append to cols and to data
dataset_cols = training.meta_data['final_dataset']['columns']
dataset_cols.extend(['frontal_suptent_ratio', 'frontal_cortex_ratio'])
train_data = np.append(train_data, np.hstack((frontal_suptent_train, frontal_cortex_train)), axis=1)
val_data = np.append(val_data, np.hstack((frontal_suptent_val, frontal_cortex_val)), axis=1)

In [5]:
svr = SVR()
svr.fit(train_data, training.labels)
preds= svr.predict(val_data)
mse(preds, validation.labels)

72.64692575246877

In [6]:
#here think about how we will benchmark the data
models = {
    'majority': np.array([np.mean(training.labels) for x in val_data]), 
    'random': np.random.rand(len(val_data))*15, #15 is a sd of our distribution
    'svr': SVR(),
    'randfor': RandomForestRegressor(), 
    'xgboost': xgboost.XGBRegressor()
}
#list - each element is an object of a model that we want to try


def restrict_dataset(final_cols, df_dict, filenames, include_generated=True):
    #here we restrict our dataset to only columns that are found in the dataframes from the following filenames
    valid_cols = []
    if include_generated:
        valid_cols.extend(['frontal_suptent_ratio', 'frontal_cortex_ratio'])
    for file in filenames:
        valid_cols.extend(df_dict[file]['columns'])
    valid_cols = set(final_cols).intersection(set(valid_cols))
    valid_inds = np.array([i for i,x in enumerate(final_cols) if x in valid_cols])
    return valid_inds


In [7]:
import itertools
files_to_explore = ['./data/btsv01.txt','./data/cc_training_v2.csv' , 'data/training_data_means.csv', \
                    'data/training_data_stdevs.csv', 'data/training_data_entropy.csv']
files_compound_variants = [list(set(x)) for x in itertools.permutations(files_to_explore, len(files_to_explore))]
# training.meta_data.keys()
column_variants = [restrict_dataset(dataset_cols, training.meta_data, f) for f in files_compound_variants]
column_variants.extend([restrict_dataset(dataset_cols, training.meta_data, f, include_generated=False) for f in files_compound_variants])
column_variants.append([x for x in range(train_data.shape[1])])

In [8]:
from IPython.display import display, clear_output
import time
results = {
  'majority': mse(models['majority'], validation.labels), 
    'random': mse(models['random'], validation.labels)
}
best_val = 100
best_ind = 0
for i,x in enumerate(column_variants):
    results[i] = {}
    for model in models.keys(): 
        if model not in ['majority', 'random']:
            models[model].fit(train_data[:,x], training.labels)
            pred = models[model].predict(val_data[:,x])
            results[i][model] = mse(pred, validation.labels)
            if results[i][model] < best_val: 
                best_val = results[i][model]
                best_ind = i
    clear_output(wait=True)
    display('{} completed'.format(i))
    time.sleep(0.01)

'4 completed'

In [9]:
best_val

72.59898973466802

In [10]:
int_to_cols = {i: x for i,x in enumerate(files_compound_variants)}

In [11]:
import pickle

results_file = 'data/results_2.pkl'
with open(results_file, 'wb') as f:
    pickle.dump(results, f)
    
int_to_col_file = 'data/variants_mapping_2.pkl'
with open(int_to_col_file, 'wb') as f: 
    pickle.dump(int_to_cols, f)


In [11]:
svr_results = [results[x]['svr'] for x in results.keys() if x not in ['random', 'majority']]
randfor_results = [results[x]['randfor'] for x in results.keys() if x not in ['random', 'majority']]
xgb_results = [results[x]['xgboost'] for x in results.keys() if x not in ['random', 'majority']]

In [12]:
print(np.mean(svr_results))
print(np.mean(randfor_results))
print(np.mean(xgb_results))

72.62775134534847
78.98212569529544
73.3435339588165


In [13]:
best_clf = SVR()
best_clf.fit(train_data[:,column_variants[best_ind]], training.labels)
preds_train = best_clf.predict(train_data[:,column_variants[best_ind]])
preds_val = best_clf.predict(val_data[:,column_variants[best_ind]])
print("MSE on train: {}".format(mse(preds_train, training.labels)))
print("MSE on val: {}".format(mse(preds_val, validation.labels)))

MSE on train: 75.06289660069592
MSE on val: 72.59898973466802


In [14]:
import eli5
from eli5.sklearn import PermutationImportance

In [15]:
perm = PermutationImportance(best_clf, random_state=1, scoring='neg_mean_squared_error').fit(val_data[:,column_variants[best_ind]], validation.labels)


In [16]:
eli5.show_weights(perm, feature_names = np.array(dataset_cols)[column_variants[best_ind]])

Weight,Feature
0.2923 ± 0.1954,sri24amygdalalgm
0.1739 ± 0.1577,sri24cerebelum6rvolume
0.1582 ± 0.1738,sri24cerebelum6lvolume
0.1417 ± 0.0579,C3
0.1299 ± 0.0886,sri24insulargm
0.1182 ± 0.2366,sri24temporalinflgm
0.1156 ± 0.1657,sri24parietalinfrgm
0.1133 ± 0.0417,CC_perimeter
0.1113 ± 0.1100,sri24rolandicoperrgm
0.1093 ± 0.0952,W7


In [17]:
len(perm.feature_importances_)
df_feature_imp = pd.DataFrame(columns=['feature_name', 'feature_importance'])
df_feature_imp['feature_name'] = np.array(dataset_cols)[column_variants[best_ind]]
df_feature_imp['feature_importance'] = perm.feature_importances_

In [19]:
file_feature_import= 'data/feature_importance_2.pkl'
with open(file_feature_import, 'wb') as f: 
    pickle.dump(df_feature_imp, f)

In [32]:
n_best = len(np.where(df_feature_imp['feature_importance'] > 0)[0])
print(n_best)
train_restricted = train_data[:, np.argsort(-df_feature_imp['feature_importance'])[:n_best]]
best_clf.fit(train_restricted, training.labels)

75


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [33]:
rsts_preds = best_clf.predict(val_data[:, np.argsort(-df_feature_imp['feature_importance'])[:n_best]])
print(mse(rsts_preds, validation.labels))

68.5445253529614


In [64]:
df_feature_imp.iloc[np.argsort(-df_feature_imp['feature_importance'].values)[:n_best]]

,feature_name,feature_importance
344,e71,0.187518
61,sri24parietalinflgm,0.109261
92,sri24cerebelumcrus1rvolume,0.093824
60,sri24parietalsuprgm,0.090792
110,sri24vermis2gm,0.080968
478,s70,0.079129
99,sri24cerebelum6lvolume,0.075897
41,sri24amygdalalgm,0.071901
305,e32,0.071264
70,sri24paracentrallobulergm,0.068391
